In [1]:
import sys
import spotipy
import spotipy.util as util
import pandas as pd
import numpy as np
sp = spotipy.Spotify()

def get_playlist_tracks(credentials,username,playlist_id):
    #set scope to retreive public data
    scope_playlist = 'playlist-modify-public'
    #provide token using given credentials
    #credentials are: username, public key, private key and redirect_uri
    token = util.prompt_for_user_token(credentials[0],scope_playlist,credentials[1],credentials[2],credentials[3])
    sp = spotipy.Spotify(auth=token)
    #get tracklisting from playlist data
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    #ensure collection of data after more than 100 requests
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

def get_features(tracks):
    #set scope to retreive public data
    scope_playlist = 'playlist-modify-public'
    #provide token using given credentials
    #credentials are: username, public key, private key and redirect_uri
    token = util.prompt_for_user_token(credentials[0],scope_playlist,credentials[1],credentials[2],credentials[3])
    sp = spotipy.Spotify(auth=token)
    #make empty list to collect dictionaries for dataframe
    dataframe = []
    for x in tracks:
        try:
            track = x['track']
            #get audio features per track
            feat = (sp.audio_features(tracks=[track['id']])[0])
            #get track and artist info
            art_dict = {'artist':track['artists'][0]['name'], 'title':track['name'], 'popularity':track['popularity']}
            #combine both dictionaries
            art_dict.update(feat)
            #make dataframe from dictionary
            dataframe.append(art_dict)
        except:
            print("Oh no!",sys.exc_info()[0],"happend")
    #filter dataframe to neccesary columns
    df = pd.DataFrame(dataframe)
    df = df.filter(['title', 'artist', 'danceability','energy', 'loudness', 'tempo','valence', 'popularity','id'])
    df = df.set_index('title')
    print("Finished collecting Songs")
    return df

def add_tracks_from_df(credentials,playlist,dataframe,features):
    scope_playlist = 'playlist-modify-public'
    token = util.prompt_for_user_token(credentials[0],scope_playlist,credentials[1],credentials[2],credentials[3])
    sp = spotipy.Spotify(auth=token)
    #make selection based on feature input
    dataframe = dataframe.drop_duplicates()
    selection = dataframe.loc[(dataframe['danceability'].between(features[0]-0.1, features[0]+0.1, inclusive=False)) &\
    (dataframe['energy'].between(features[1]-0.2, features[1]+0.2, inclusive=False)) &\
    (dataframe['loudness'].between(features[2]-5, features[2]+5, inclusive=False)) &\
    (dataframe['tempo'].between(features[3]-40, features[3]+40, inclusive=False)) &\
    (dataframe['valence'].between(features[4]-0.5, features[4]+0.5, inclusive=False))] 
    #remove unwanted artists
    # & (~dataframe['artist'].isin(['Ariana Grande']))]
    selection.sort_values(by=['popularity'], ascending=False)
    songs = selection['id'].tolist()
    
    if len(songs) > 0:
        scope_playlist = 'playlist-modify-public'
        token = util.prompt_for_user_token(credentials[0],scope_playlist,credentials[1],credentials[2],credentials[3])
        sp = spotipy.Spotify(auth=token)
        sp.user_playlist_add_tracks(credentials[0],playlist, songs[:100])
        print("Songs Succesfully Added to Playlist")
    else:
        print("There are no suitable songs in your collection")
    
def remove_tracks_from_df(credentials,playlist,dataframe,features):
    scope_playlist = 'playlist-modify-public'
    token = util.prompt_for_user_token(credentials[0],scope_playlist,credentials[1],credentials[2],credentials[3])
    sp = spotipy.Spotify(auth=token)
    dataframe = dataframe.drop_duplicates()
    selection = dataframe.loc[(dataframe['danceability'].between(features[0]-0.1, features[0]+0.1, inclusive=False)) &\
    (dataframe['energy'].between(features[1]-0.2, features[1]+0.2, inclusive=False)) &\
    (dataframe['loudness'].between(features[2]-5, features[2]+5, inclusive=False)) &\
    (dataframe['tempo'].between(features[3]-40, features[3]+40, inclusive=False)) &\
    (dataframe['valence'].between(features[4]-0.5, features[4]+0.5, inclusive=False))]
    selection.sort_values(by=['popularity'], ascending=False)
    songs = selection['id'].tolist()
    
    scope_playlist = 'playlist-modify-public'
    token = util.prompt_for_user_token(credentials[0],scope_playlist,credentials[1],credentials[2],credentials[3])
    sp = spotipy.Spotify(auth=token)
    sp.user_playlist_remove_all_occurrences_of_tracks(credentials[0],playlist, songs[:100])
    print("Songs Succesfully Removed from Playlist")
    
def update_store_list(credentials,playlist,dataframe,features):
    scope_playlist = 'playlist-modify-public'
    token = util.prompt_for_user_token(credentials[0],scope_playlist,credentials[1],credentials[2],credentials[3])
    sp = spotipy.Spotify(auth=token)
    store_list = get_playlist_tracks(credentials,'bartw26396','5yJfsUa3aWq20QhPLGwtig')
    if len(store_list) == 0:
        print('Updating Playlist...')
        add_tracks_from_df(credentials,playlist,dataframe,features)
    else:
        songs =[]
        for x in list_length:
            songs.append(x['track']['id'])
        print('Clearing Playlist...')
        sp.user_playlist_remove_all_occurrences_of_tracks(credentials[0],playlist, songs[:100])
        print("Songs Succesfully Removed from Playlist")
        print('Updating Playlist...')
        add_tracks_from_df(credentials,playlist,dataframe,features)


In [2]:
credentials = ['bartw26396', '5711bc132b4c48ceb5bbd19cd65b1e63', 'f507991961c948d8bf1b62ae6ef5ab15', 'http://localhost']
playlists = {'Kasper Langendoen':'4W7jnrqeKfVEnb1BVHMG5b', 'Top 50 Wereld':'37i9dQZEVXbMDoHDwVN2tF', 'NPO Radio 2':'1DTzz7Nh2rJBnyFbjsH1Mh',\
'daryl zandvliet':'6PoHyrIELxnRlRKOsI5yhW', 'Slam Official':'0OdWlUFdB6Lio5dIdXY81O', 'Bouke Bosma':'70aT8IllF7t6CLcPf2pt99'}

datafroem = pd.DataFrame()
for key, value in playlists.items():
    tracklist = get_playlist_tracks(credentials, key,value)
    data = get_features(tracklist)
    datafroem = datafroem.append(data)
    


retrying ...4secs
Oh no! <class 'KeyboardInterrupt'> happend
retrying ...7secs
Oh no! <class 'KeyboardInterrupt'> happend
Oh no! <class 'KeyboardInterrupt'> happend
Oh no! <class 'KeyboardInterrupt'> happend
retrying ...7secs
Oh no! <class 'KeyboardInterrupt'> happend
retrying ...6secs
Oh no! <class 'KeyboardInterrupt'> happend
retrying ...5secs
Finished collecting Songs
retrying ...7secs
retrying ...7secs
Finished collecting Songs
retrying ...5secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...6secs
retrying ...7secs
retrying ...6secs
retrying ...7secs
retrying ...6secs
retrying ...7secs
retrying ...6secs
retrying ...6secs
retrying ...8secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...7secs
retrying ...8secs
retrying ...6secs
retrying ...7secs
retrying ...7secs
retrying ...8secs
retrying ...7secs
retrying ...7secs
retrying ...6secs
retrying ...6secs
retrying ...6secs
retrying ...7secs
Finished collecting Song

In [4]:
features = [0.615,0.615,-5.4,98.0,0.36]
update_store_list(credentials,'5yJfsUa3aWq20QhPLGwtig',datafroem,features)

NameError: name 'list_length' is not defined

In [3]:
datafroem

,artist,danceability,energy,loudness,tempo,valence,popularity,id
title,,,,,,,,
Keep Control - ARTBAT Remix,Sono,0.769,0.762,-9.669,123.002,0.119,43,6n8wkmgsU4wCJdM2rky0yb
Eyes On Fire - Michael Bibi Remix Edit,Blue Foundation,0.800,0.683,-8.771,127.995,0.625,47,5kNOW0qMt6XN4d4rQUUu03
Remember Me - Franky Rizardo Radio Edit,Blue Boy,0.835,0.898,-7.126,127.989,0.925,41,5pUvWWkn5yONeE8DWBTrgC
DT64,Moguai,0.743,0.945,-8.660,129.992,0.172,57,5z7VT7KoeFLT1rjeyDzdMI
Deadeye,Sorley,0.810,0.915,-7.618,124.994,0.741,45,5o7M979CiUYnY1M2l6VMQ0
...,...,...,...,...,...,...,...,...
You're The One That I Want - From “Grease”,John Travolta,0.758,0.612,-11.403,106.945,0.822,74,0B9x2BRHqj3Qer7biM3pU3
Mr. Jones,Counting Crows,0.578,0.817,-6.542,141.621,0.726,75,5DiXcVovI0FcY2s0icWWUu
Times Are Changing,DI-RECT,0.501,0.810,-6.106,122.968,0.285,47,2ChLCoNCOIXSKEgrRueGsy
